# Mục lục

File notebook này được viết trong `google colab`, để đạt được đảm bảo về `format` và trải nghiệm khi đọc, việc mở trong [google colab](https://drive.google.com/file/d/1QHz8wXTMAjqHkHsLgOlhd5nniCsUwfjU/view?usp=sharing) được khuyến khích.

>[Mục lục](#scrollTo=S0v9dmw8OaXp)

>[Lời tựa](#scrollTo=9u2Wg5kWwxjw)

>>[Thông tin chung](#scrollTo=ZkTgd6H5z0ZK)

>>[Asyncio](#scrollTo=mdkeQjwDyWcY)

>>[API](#scrollTo=AdIUUYcO337E)

>[Cài đặt các thư viện](#scrollTo=mWLilnJnrn0n)

>[Import các thư viện](#scrollTo=ymTpJuxxrtJi)

>[Set up các biến cục bộ](#scrollTo=eIH5S5wzsErA)

>>[Logger](#scrollTo=2Fn-6yngvv4y)

>[Nhóm các hàm chung](#scrollTo=T3KC4aOboqhU)

>>[Hàm save_file](#scrollTo=OhCbXb0Ro_dY)

>>[Hàm join_list](#scrollTo=Mck8J6JBqphU)

>>[Hàm request_url](#scrollTo=iQfqNFcT015P)

>>[Hàm get_id_from_collection](#scrollTo=as2X8ueO7RL6)

>>[Nhóm hàm tìm thông tin của 1 track](#scrollTo=RUqfXvZBIhgk)

>>>[Hàm extract_track_generals](#scrollTo=aUYc43mmIxf_)

>>>[Hàm track_info](#scrollTo=OT5GplGOI8if)

>>[Nhóm hàm tìm thông tin của 1 playlist](#scrollTo=uwJEXcx5NuCa)

>>>[Hàm extract_playlist_generals](#scrollTo=EchJKg-cYdXl)

>>>[Hàm playlist_info](#scrollTo=opDOeTjegzWE)

>>[Hàm lấy thông tin user](#scrollTo=leVtCGXviZCm)

>[Hàm lấy thông tin từ keyword](#scrollTo=r6esDxz-j-Eb)

>[Hàm lấy thông tin tổng quát](#scrollTo=lBX66bj7qd1A)

>>[Hàm lấy thông tin Featured tracks](#scrollTo=DdX0h6UnrXwc)

>>[Nhóm hàm lấy thông tin từ Charts tracks](#scrollTo=Ogl2FH3qtuGc)

>>>[Hàm extract_charts_data](#scrollTo=aSJW7zLet3t6)

>>>[Hàm get_charts](#scrollTo=1jWD3-jBubAG)

>>[Nhóm hàm lấy thông tin từ discover tracks and playlists](#scrollTo=IgBNOcUkwuvd)

>>>[Hàm get_discover_id](#scrollTo=4J02rxTyw6Qo)

>>>[Hàm get_discover_tracks](#scrollTo=xsZvrx50yAnS)

>>>[Hàm get_discover_playlists](#scrollTo=zE6P_aWozDLN)

>>>[Hàm get_discover](#scrollTo=ZIiPbA94zz9_)

>[Nhóm hàm thực thi chương trình](#scrollTo=sWCDRqBC4S6S)

>>[Hàm extract_data](#scrollTo=qRYDP2KZ4c44)

>>[Hàm main](#scrollTo=NfvurLj05g1q)

>>[Chạy chương trình](#scrollTo=nHYOH39y8EqP)

>>>[Window](#scrollTo=uCXdWQym8ICZ)

>>>[Linux](#scrollTo=gFwVPLxn8d84)

>>>[Jupyter](#scrollTo=SqPVyxwO82T1)



# Lời tựa

## Thông tin chung

File notebook này chỉ trình bày về các `function` và cách thức hoạt động cơ bản của các hàm trong chương trình.

Thông tin chi tiết về API sử dụng, quá trình thực hiện và kết quả thực hiện sẽ thể hiện qua báo cáo và `API Document` (file khác)



## Asyncio

Để tận dụng khoảng thời gian chờ lúc request, trong chương trình này ta sẽ sử dụng thư viện `asyncio` và kỹ thuật lập trình `multithreading` để định nghĩa các `couroutine`, với mục đích hướng đến việc chạy các `function` song song (hoặc chí ít là trông có vẻ như vậy), điều này sẽ tiết kiệm đáng kể tổng thời gian thực hiện chương trình. 

Hướng dẫn về kĩ thuật này có thể được tìm thấy ở đây:
- [Official Documentations](https://docs.python.org/3/library/asyncio.html) 
- [Async IO in Python: A Complete Walkthrough](https://realpython.com/async-io-python/)

## API 

Để hỗ trợ cho quá trình làm đồ án, nhóm cũng đã thực hiện viết document cho API-v2 của SoundCloud.

Bởi 'v2 API' của SoundCloud là version không có public document, không được khuyến khích sử dụng bởi SoundCloud, nhưng lại có các tính năng mà public API (`v1`) còn thiếu.

File Document này được tách thành file riêng và đính kèm trong bài nộp. 

# Cài đặt các thư viện

In [ ]:
!pip install aiohttp

# Import các thư viện

In [ ]:
from enum import Enum
import asyncio
import csv
import aiohttp
from datetime import datetime
import logging


# Set up các biến cục bộ

## Logger

Để thuận tiện cho việc khảo sát, debug, theo dõi tiến trình của chương trình, ta sẽ sử dụng thư viện `logging` để tiến hành `log` kết quả lên màn hình, cũng như `log` vào file `*.log` nào đó. 

In [ ]:
logFormatter = logging.Formatter("%(asctime)s [%(levelname)-5.5s]  %(message)s")

`logFormatter` là biến lưu giữ format tổng quát của 1 dòng thông báo

In [ ]:
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.DEBUG)

`rootLogger` sẽ là handle chính mà ta có thể gọi hàm lưu trữ hoặc in ra console, với `level` là `DEBUG`. Điều này có nghĩa bất cứ các `log` nào có mức `level` là DEBUG trở lên sẽ được ghi nhận.

Từ handle chính này ta sẽ thêm 2 handle phụ, 1 cái là để `log` lên console, 1 cái là để `log` vào file.

In [ ]:
fileHandler = logging.FileHandler(filename='crawler.log', mode='w')
fileHandler.setFormatter(logFormatter)
fileHandler.setLevel(logging.DEBUG)
rootLogger.addHandler(fileHandler)

Cần lưu ý `level` của `fileHandler` là `DEBUG`, tức chỉ những `log` có mức độ DEBUG trở lên mới được ghi vào file `"crawler.log"`.

In [ ]:

consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logFormatter)
consoleHandler.setLevel(logging.INFO)
rootLogger.addHandler(consoleHandler)


Thông báo `log` lên console thì không cần thiết phải chi tiết đến vậy, ta chỉ cần `log` những thông báo có `level` từ `INFO` trở lên.

Lưu ý với mỗi handle đã định nghĩa ở trên, ta cần thêm chúng vào `rootLogger` với hàm `addHandler`.

Chi tiết sử dụng các hàm trong thư viện này có thể tìm được ở:
- [Official Documentations](https://docs.python.org/3/library/logging.html) 
- [Logging in Python](https://realpython.com/python-logging/)

# Nhóm các hàm chung 

## Hàm `save_file`

In [ ]:
def save_file(file_name, json_data):
    keys = set()
    for d in json_data:
        keys.update(d.keys())
    keys = sorted(keys)
    with open(file_name, 'w', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, restval="-", fieldnames=keys, delimiter='\t')
        dict_writer.writeheader()
        dict_writer.writerows(json_data)

Hàm `save_file` có chức năng chuyển json data  truyền vào thành file csv

- `file_name`: parameter có dạng string, là tên của file data (ex: `"users.csv"`)
- `json_data`: là 1 list mà từng phần tử của nó chứa các `dictionary` với `key` là các `field` dữ liệu và `value là` `giá trị` của các `field` này.
- `save_file`: sẽ xuất ra một file chứa dữ liệu với các parameter đã cung cấp.

## Hàm `join_list`

In [ ]:
async def join_list(l):
    values = ','.join([str(i) for i in l])
    return values

Đây là 1 `coroutine`.


Hàm `join_list` trả về 1 chuỗi các phần tử của list và được phân cách bởi dấu `,`

Các giá trị trả về từ API nhiều lúc xuất hiện ở dạng `list`, để thuận tiện cho việc lưu vào file csv ta cần chuyển `list` này thành 1 string.


- `l`: parameter nhận vào 1 `list`
- `join_list`: trả về chuỗi string sau khi `join` chúng lại với nhau.




## Hàm `request_url`

In [ ]:
async def request_url(url, max_req=10):
    req = 0
    try:
        async with aiohttp.ClientSession() as session:
            while True:
                async with session.get(url) as response:
                    if response.status != 200:
                        rootLogger.debug(f"FAILED {response.url}")
                        req += 1
                        if req < max_req:
                            continue
                        else:
                            rootLogger.info(f'ABORTED url: {response.url}')
                            return {}
                    rootLogger.debug(f"HIT {response.url}")
                    return await response.json()
    except Exception as e:
        rootLogger.error(f'{e}')
        rootLogger.debug(f"REQUEST_URL NEXT ATTEMPT on {url}")
        return await request_url(url, max_req)

Đây là 1 `coroutine`.


Hàm `request_url` sử dụng `method` `get()` của thư viện `aiohttp` để thực hiện gửi yêu cầu cho SoundCloud API server và trả về kết quả dưới định dạng json.

`URL` dùng để request có thể trả về kết quả với status code khác `200`. Cụ thể ở đây là `404` hoặc `403`, với những trường hợp này việc request lại có khả năng trả về kết quả `200` (đã kiểm chứng). Tuy nhiên ta không thể request lại mãi, việc đưa 1 biến định nghĩa số lần tối đa request trên 1 url cho trước là cần thiết. Nếu quá giới hạn này mà kết quả trả về vẫn file, ta kết luận không thể lấy được kết quả và trả về `rỗng`.

Do lỗi của thư viện `aiohttp` hoặc do SoundCloud API server. Trong quá trình thực hiện chương trình sẽ phát sinh ra các lỗi như: `connection reset by peer`, `hostname not available`,... hướng xử lý lỗi này khá đơn giản, ta chỉ cần gọi lại hàm `request_url` để `aiohttp session` thiết lập lại kết nối.


- `url`: là url truyền vào để lấy dữ liệu từ server, chi tiết tham khảo API document của nhóm.
- `max_req`: biến này quy định số lần tối đa mà 1 url có thể request, ở đây mặc định là 10.
- `request_url`: trả về 1 dictionary với các cặp key-value tương ứng với kết quả trả về của API server, trong trường hợp request thất bại, hàm trả về dictionary rỗng `{}`.

## Hàm `get_id_from_collection`

In [ ]:
async def get_id_from_collection(url, client_id, result_limit, option=None):
    results = []
    full = False
    while True:
        json_data = await request_url(url)
        try:
            collections = json_data['collection']
            if not collections:
                break
            for collection in collections:
                if len(results) < result_limit or result_limit == -1:
                    if option:
                        results.append(collection[option]['id'])
                    else:
                        results.append(collection['id'])
                else:
                    full = True
                    break
            if full or json_data['next_href'] is None:
                break
            url = json_data['next_href'] + f'&client_id={client_id}'
        except KeyError:
            break
    return list(set(results))


Đây là 1 `coroutine`.

Do cấu trúc dữ liệu trả về đặc thù của `SoundCloud API`, rất thường xuyên có cấu trúc `json` chứa các `id` nằm trong tập collection, cụ thể như sau:

```JSON
{
  collection: [
    {"id": "1234"},
    {"id": "5678"},
    "next_href": <next_url>
    ...
  ]
}
```

Hoặc
```JSON
{
  collection: [
    {
    "option": {"id": "1234"}
    },
    {
    "option": {"id": "5678"}
    },
    "next_href": <next_url>
    ...
  ]
}
```

Bởi cấu trúc này hàm `get_id_from_collection` là cần thiết cho việc trích xuất, mục đích của hàm là lấy các `id` từ cấu trúc dữ liệu trên.

Rất thường xuyên, các dữ liệu này không được trả về hết, mà trả về theo "trang", để lấy được số `id` mong muốn, ta cần "lật" qua một trang mới, sau đó chạy lại hàm để trích `id`. `URL` của "trang" mới này sẽ được lưu trữ trong trường `next_href`.

Vậy hàm này chạy theo các bước:
1. Request `trang` hiện tại và lấy các `id` liên quan
2. Lật qua trang mới để thu thập tiếp.

- `url`: là url đưa vào để request, request này sẽ trả về cấu trúc dữ liệu trên
- `client_id`: các request đến server của SoundCloud API sẽ cần client_id như 1 parameter trong url (xem chi tiết API-v2 document)
- `result_limit`: là biến có vai trò giới hạn kết quả trả về, đôi khi ta không cần lấy hết tất cả các `id` trong dữ liệu mà chỉ cần 1 số nào đó. Nếu parameter này là `-1`, chương trình sẽ cố lấy hết tất cả `id`.
-  `option=None`: với `None` là giá trị mặc định, làm này sẽ trích ra cấu trúc dữ liệu trong ví dụ thứ 2 đề cập ở trên. Cụ thể `collection["option"]["id"]`

- `get_id_from_collection`: trả về 1 `list` các id từ url truyền vào.

## Nhóm hàm tìm thông tin của 1 `track`

### Hàm `extract_track_generals`


In [ ]:
async def extract_track_generals(track_id, client_id):
    url = f'https://api-v2.soundcloud.com/tracks/{track_id}?client_id={client_id}&linked_partitioning=1'
    generals_data = await request_url(url)
    if generals_data:
        generals_data['user'] = generals_data['user']['id']
    return generals_data

Đây là 1 `coroutine`.

Hàm `extract_track_generals` sẽ gởi request tới SoundCloud API server và trả về kết quả thu thập được.

Đồng thời cấu trúc lại dữ liệu 1 chút, cụ thể là trích `id` của người đăng `track` và gán lại vào field liên quan. Lúc này:
```JSON
{
  "user": {"id": "1234"}
}
``` 
Sẽ trở thành:
```JSON
{
  "user": "1234"
}
```

Điều này là cần thiết để thuận tiện cho việc lưu trữ dữ liệu vào file `csv`. Kĩ thuật rút trích, hoặc lược bỏ này được sử dụng xuyên suốt các hàm trích `info`.

- `track_id`: `id` cụ thể của 1 `track` nào đó, đây là `track` ta muốn trích xuất dữ liệu.
- `client_id`: client_id của user, parameter này là cần thiết trong `url` dùng để request.
- `extract_track_generals`: sẽ trả về dữ liệu "tổng quan" của 1 `track` với `track_id` đã định nghĩa từ trước.

### Hàm `track_info`


In [ ]:
async def track_info(track_id, client_id):
    reposters_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/reposters?client_id={client_id}&limit=100&linked_partitioning=1'
    likers_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/likers?client_id={client_id}&limit=100&linked_partitioning=1'
    related_tracks_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/related?client_id={client_id}&limit=100&linked_partitioning=1'
    album_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/albums?&client_id={client_id}&limit=100&linked_partitioning=1'
    playlists_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/playlists_without_albums?client_id={client_id}&limit=100&linked_partitioning=1'
    comments_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/comments?threaded=0&filter_replies=0&client_id={client_id}&limit=100&linked_partitioning=1'

    generals_data = await extract_track_generals(track_id, client_id)
    if generals_data:
        sub_info = await asyncio.gather(
            join_list(await get_id_from_collection(reposters_url, client_id, 100)),
            join_list(await get_id_from_collection(likers_url, client_id, 100)),
            join_list(await get_id_from_collection(comments_url, client_id, 100, 'user')),
            join_list(await get_id_from_collection(related_tracks_url, client_id, 100)),
            join_list(await get_id_from_collection(album_url, client_id, 100)),
            join_list(await get_id_from_collection(playlists_url, client_id, 100)),
        )
        generals_data['reposters'] = sub_info[0]
        generals_data['likers'] = sub_info[1]
        generals_data['comments'] = sub_info[2]
        generals_data['related_tracks'] = sub_info[3]
        generals_data['album'] = sub_info[4]
        generals_data['playlists'] = sub_info[5]

        generals_data.pop('publisher_metadata')
        generals_data.pop('media')
        generals_data.pop('visuals')
    return generals_data

Đây là 1 `coroutine`.

Hàm lấy về các thông tin còn lại của `track`, tiếp nối hàm `extract_track_generals`.

Gồm 2 bước:
1. Thông qua `extract_track_generals` để lấy thông tin tổng quan
2. Sử dụng các `API url` liên quan để lấy thông tin phụ

Các `url` về các dữ liệu liên quan lần lược được sử dụng với `track_id` và `client_id` làm parameter. Các trả về của trường này thuộc cấu trúc "trang" đã đề cập, nên ta sẽ dùng `get_id_from_collection` để lấy dữ liệu về. 

Ở đây mỗi trường dữ liệu ta lấy `100` kết quả, sau đó biến chúng thành 1 string với nhau hàm `join_list`.

Kết quả cuối cùng được lược bỏ bớt các field không cần thiết hoặc có cấu trúc phức tạp nhưng lại không mang nhiều ý nghĩa (với người thu thập).

Ở đây cũng có thể lưu ý rằng tất cả các tiến trình thu thập thông tin phụ này đều được chạy song song với nhau thông qua `asyncio.gather` để tiết kiệm thời gian và làm code nhanh hơn. Đây cũng là kĩ thuật được sử dụng xuyên suốt trong các hàm request data phụ.

- `track_id`: là `id` cụ thể của `track` cần lấy thông tin
- `client_id`: là `id` của client, hay người thực hiện requests.
- `track_info`: trả về 1 `dictionary` chứa thông tin về `track` truyền vào

## Nhóm hàm tìm thông tin của 1 `playlist`

### Hàm `extract_playlist_generals`

In [ ]:
async def extract_playlist_generals(playlist_id, client_id):
    url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}?client_id={client_id}&limit=100&linked_partitioning=1'
    generals_data = await request_url(url)
    if generals_data:
        generals_data['user'] = generals_data['user']['id']
        tracks_list = []
        for track in generals_data['tracks']:
            tracks_list.append(track['id'])
        generals_data['tracks'] = list(set(tracks_list))
    return generals_data

Đây là 1 `coroutine`.

Hàm tương tự hàm `extract_track_generals` đã đề cập ở trên, `extract_playlist_generals` cũng gởi request tới SoundCloud API server và lấy về thông tin tổng quan về 1 `playlist` với `id` của `playlist` đó cho trước.

Sau đó hàm sẽ cấu trúc lại một số trường dữ liệu, cụ thể là trường `user` để lấy `id` và `tracks` để lấy các `track` thuộc `playlist` này, trường dữ liệu "tracks" sẽ là `list` các `id` của các `track`.

- `playlist_id`: `id` của `playlist` cần thu thập thông tin.
- `client_id`: `id` của client để thực hiện request.
- `extract_playlist_generals`: trả về thông tin tổng quan của `playlist`.

### Hàm `playlist_info`

In [ ]:
async def playlist_info(playlist_id, client_id):
    reposters_url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}/reposters?client_id={client_id}&limit=100&linked_partitioning=1'
    likers_url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}/likers?client_id={client_id}&limit=100&linked_partitioning=1'
    generals_data = await extract_playlist_generals(playlist_id, client_id)
    if generals_data:
        sub_info = await asyncio.gather(
            join_list(await get_id_from_collection(reposters_url, client_id, 100)),
            join_list(await get_id_from_collection(likers_url, client_id, 100)),
            join_list(generals_data['tracks'])
        )
        generals_data['reposters'] = sub_info[0]
        generals_data['likers'] = sub_info[1]
        generals_data['tracks'] = sub_info[2]
    return generals_data

Đây là 1 `coroutine`.

Sau khi có được thông tin tổng quan của `playlist` có được ở `extract_playlist_generals`, ta lấy thông tin phụ của playlist này qua các hàm `get_id_from_collection` và `join` chúng lại với nhau thành 1 `string`.

Sử dụng `asyncio.gather`, ta chạy được song song các tiến trình thu thập thông tin phụ này.

Vậy có 2 bước chính:
1. Thông qua `extract_playlist_generals` để tìm các thông tin tổng quan
2. Sử dụng các `API url` liên quan để lấy thông tin phụ

- `playlist_id`: `id` của playlist cần lấy thông tin.
- `client_id`: `id` của client để thực hiện request.
- `playlist_info`: trả về 1 `dictionary` mà mỗi cặp `key` - `value` tương ứng với các `field` của dữ liệu trả về từ SoundCloud API server. 

## Hàm lấy thông tin `user`

In [ ]:
async def user_info(user_id, client_id):
    general_url = f'https://api-v2.soundcloud.com/users/{user_id}?client_id={client_id}&limit=100&linked_partitioning=1'
    web_profile_url = f'https://api-v2.soundcloud.com/users/soundcloud:users:{user_id}/web-profiles?client_id={client_id}'
    spotlight_url = f'https://api-v2.soundcloud.com/users/{user_id}/spotlight?client_id={client_id}&limit=100&linked_partitioning=1'
    user_tracks_url = f'https://api-v2.soundcloud.com/users/{user_id}/tracks?client_id={client_id}&limit=100&linked_partitioning=1'
    user_top_tracks_url = f'https://api-v2.soundcloud.com/users/{user_id}/toptracks?client_id={client_id}&linked_partitioning=1'
    user_albums_url = f'https://api-v2.soundcloud.com/users/{user_id}/albums?client_id={client_id}&limit=100&linked_partitioning=1'
    user_playlist_without_albums_url = f'https://api-v2.soundcloud.com/users/{user_id}/playlists_without_albums?client_id={client_id}&limit=100&linked_partitioning=1'
    related_artist_url = f'https://api-v2.soundcloud.com/users/{user_id}/relatedartists?client_id={client_id}&limit=100&linked_partitioning=1'
    reposts_url = f'https://api-v2.soundcloud.com/stream/users/{user_id}/reposts?client_id={client_id}&limit=100&linked_partitioning=1'
    likes_url = f'https://api-v2.soundcloud.com/users/{user_id}/likes?client_id={client_id}&limit=100&linked_partitioning=1'
    followings_url = f'https://api-v2.soundcloud.com/users/{user_id}/followings?client_id={client_id}&limit=100&linked_partitioning=1'
    followers_url = f'https://api-v2.soundcloud.com/users/{user_id}/followers?client_id={client_id}&limit=100&linked_partitioning=1'

    generals_data = await request_url(general_url)
    if generals_data:
        sub_info = await asyncio.gather(
            request_url(web_profile_url),
            join_list(await get_id_from_collection(spotlight_url, client_id, 100)),
            join_list(await get_id_from_collection(user_tracks_url, client_id, 100)),
            join_list(await get_id_from_collection(user_top_tracks_url, client_id, 100)),
            join_list(await get_id_from_collection(user_albums_url, client_id, 100)),
            join_list(await get_id_from_collection(user_playlist_without_albums_url, client_id, 100)),
            join_list(await get_id_from_collection(related_artist_url, client_id, 100)),
            join_list(await get_id_from_collection(followings_url, client_id, 100)),
            join_list(await get_id_from_collection(followers_url, client_id, 100)),
            join_list(await get_id_from_collection(reposts_url, client_id, 100, 'track')),
            join_list(await get_id_from_collection(likes_url, client_id, 100, 'track')),
        )
        generals_data['web_profile'] = sub_info[0]
        generals_data['spotlight_tracks'] = sub_info[1]
        generals_data['user_tracks'] = sub_info[2]
        generals_data['user_top_tracks'] = sub_info[3]
        generals_data['user_albums'] = sub_info[4]
        generals_data['user_playlist_without_albums'] = sub_info[5]
        generals_data['related_artist'] = sub_info[6]
        generals_data['followings'] = sub_info[7]
        generals_data['followers'] = sub_info[8]
        generals_data['reposts'] = sub_info[9]
        generals_data['likes'] = sub_info[10]

        generals_data.pop('creator_subscription')
        generals_data.pop('creator_subscriptions')
        generals_data.pop('visuals')
        generals_data.pop('badges')

        web_profile = generals_data.pop('web_profile', None)
        socials = []
        if web_profile:
            for social in web_profile:
                socials.append(social['url'])
        generals_data['socials'] = ','.join(socials)

    return generals_data


Đây là 1 `coroutine`.

Tương tự với `track` và `playlist`, hàm sẽ trả về các thông tin liên quan đến user.
Sau khi tiến hành lấy thông tin tổng quan (không có hàm phụ), `user_info` tiếp tục thông qua `asyncio.gather` để chạy song song các tiến trình lấy thông tin phụ.

Các thông tin phụ được lấy dựa vào hàm `get_id_from_collection` với `result_limit` của từng `field` là `100`. Sau đó `join` kết quả lại thành 1 `string`.

Ta có thể coi như có các bước chính sau dây:
1. Lấy thông tin tổng quan
2. Lấy thông tin phụ

Quá trình kết thúc với việc loại bỏ một số `field` dữ liệu và tái cấu trúc lại `field` "social".

- `user_id`: là `id` của `user` ta cần lấy thông tin.
- `client_id`: là `id` của `client` thực hiện request.
- `user_info`: trả về `dictionary` chứa các thông tin liên quan đến `user` được yêu cầu.

# Hàm lấy thông tin từ keyword

In [ ]:
class QueryType(Enum):
    USERS = 0
    TRACKS = 1
    PLAYLISTS = 2


Để thuận tiện cho việc yêu cầu kết quả trả về thuộc loại nào trong các loại: `USERS`, `TRACKS`, `PLAYLISTS`, việc tạo ra một `class enum` để phân biệt dễ dàng là cần thiết. 

In [ ]:
async def get_query_item(q_type, query, client_id, result_limit):
    rootLogger.debug(f"STARTING... getting all results of type {q_type.name} with query: '{query}'")
    results = []
    sub_url = ''
    if q_type == QueryType.USERS:
        url = f'https://api-v2.soundcloud.com/search/users?q={query}&client_id={client_id}&limit=100&linked_partitioning=1 '
        func = user_info
    elif q_type == QueryType.TRACKS:
        url = f'https://api-v2.soundcloud.com/search/tracks?q={query}&client_id={client_id}&limit=100&linked_partitioning=1'
        func = track_info
    elif q_type == QueryType.PLAYLISTS:
        url = f'https://api-v2.soundcloud.com/search/albums?q={query}&client_id={client_id}&limit=100&linked_partitioning=1'
        sub_url = f'https://api-v2.soundcloud.com/search/playlists_without_albums?q={query}&client_id={client_id}&limit=100&linked_partitioning=1'
        func = playlist_info
    else:
        raise Exception("Unsupported type")

    tasks = [get_id_from_collection(url, client_id, result_limit)]
    if sub_url:
        tasks.append(get_id_from_collection(sub_url, client_id, result_limit))

    item_ids = await asyncio.gather(*tasks)
    if sub_url:
        item_ids = item_ids[0] + item_ids[1]
    else:
        item_ids = item_ids[0]

    rootLogger.info(f"FOUND {len(item_ids)} items for type {q_type.name} with '{query}' keyword")
    for item_id in item_ids:
        result = await func(item_id, client_id)
        if result:
            results.append(result)
            rootLogger.info(f"COMPLETED {len(results)} / {len(item_ids)} - {q_type.name} - '{query}'")

        else:
            rootLogger.info(f"ABORTED 1 query item of type {q_type} with '{query}' keyword")
   
    rootLogger.info(f"FINISHED: got {len(results)} of type {q_type.name} with query: '{query}'")
    return results

Đây là 1 `coroutine`.

`get_query_item` dựa trên các url đã được định nghĩa từ trước trong `API DOCUMENT` để thực hiện quá trình `query`. Với từng loại `query` yêu cầu thuộc một trong các loại: `USER`, `TRACK`, `PLAYLIST`.

Đối với từng loại được yêu cầu, `function` để tìm thông tin cũng như `url` sẽ được gán hợp lý.

Hàm này gồm 2 bước chính có thể kể như sau:
1. Tìm tất cả `id` thuộc loại được yêu cầu, các `id` là kết quả từ việc `query`.
2. Với từng `id` tìm được, gọi hàm lấy thông tin tùy thuộc vào loại của `id` đó.

Kết quả trả về sẽ là 1 `list`, mà từng phần tử của `list` là 1 `dictionary` chứa dữ liệu tương ứng với từng `id` có được ở bước 1.

- `q_type`: là loại query, giá trị có thể thuộc 1 trong 3 trường hợp: `QueryType.USERS`, `QueryType.TRACKS`, `QueryType.PLAYLISTS`. Ta có thể chọn tương ứng với loại ta muốn.
- `query`: là keyword cần, các từ có thể được ngăn cách bằng `space`, hàm sẽ thực hiện query với loại được yêu cầu.
- `result_limit`: là giới hạn tối đa kết quả trả về từ quá trình này. Tuy nhiên ta cần lưu ý, nếu loại `query` yêu cầu là `QueryType.PLAYLISTS`, parameter `result_limit` sẽ được nhân đôi. Lý do cho chuyện này là vì `Albums` trong `SoundCloud API` tuy có `query url` các `playlist`, nhưng kết quả trả về lại có thể sử dụng `playlist_info` để trích xuất dữ liệu. Điều này làm cho khi `query` cho `playlist` thì thực chất sẽ có 2 lần chạy bước `1`: cho `playlist` và cho `albums`.
- `get_query_item`: sẽ trả về 1 `list` các `dictionary` kết quả như đã đề cập.


# Hàm lấy thông tin tổng quát

Có 3 nguồn chính ở `SoundCloud` ta có thể lấy thông tin mà không dựa vào bất kì một `keyword` cho trước nào:
1. [Featured tracks](https://soundcloud.com/)
2. [Discover tracks and playlists](https://soundcloud.com/discover)
3. [Charts tracks](https://soundcloud.com/charts)

Với mỗi nguồn trên, ta sẽ có các `API` liên quan để lấy dữ liệu (xem `API Document`) cùng với các nhóm hàm riêng biệt.

## Hàm lấy thông tin `Featured tracks`

In [ ]:
async def get_featured(client_id, result_limit=50):
    rootLogger.debug("STARTING... getting SoundCloud's featured items...")
    results = []
    url = f'https://api-v2.soundcloud.com/featured_tracks/top/all-music?linked_partitioning=1&client_id={client_id}&limit=100'
    featured_tracks = await get_id_from_collection(url, client_id, result_limit)

    rootLogger.info(f"FOUND {len(featured_tracks)} featured tracks")

    for track_id in featured_tracks:
        ti = await track_info(track_id, client_id)
        if ti:
            results.append(ti)
            rootLogger.info(f"COMPLETED {len(results)} / {len(featured_tracks)} featured tracks")
        else:
            rootLogger.info(f'ABORTED 1 featured track')

    rootLogger.info(f"FINISHED got {len(results)} featured tracks")
    return results

Đây là 1 `coroutine`.

Hàm lấy thông tin về các `track` trong [Featured tracks](https://soundcloud.com/) đã đề cập ở trên. Đây cũng là trang chủ của `SoundCloud`.

Có 2 bước thực hiện chính:
1. Lấy hết các `id` của các `track` trong trang này dựa trên `API` thích hợp và hàm `get_id_from_collection`.
2. Với từng `id` có được, gọi hàm `track_info` tương ứng để lấy kết quả.

- `client_id`: là `id` của `client` thực hiện request
- `result_limit`: là giới hạn tối đa kết quả trả về từ trang trên. Kết quả ở trang này thường là dưới `50` `id`, cũng có thể đây là số lượng `SoundCloud` quy định, việc parameter `result_limit` tuy không thực sự cần thiết ở đây, nhưng trong quá trình viết code lại dùng nhiều để `debug`, hơn nữa cũng để tiện kiểm soát số lượng kết quả, việc thêm vào có lợi hơn là có hại. 

## Nhóm hàm lấy thông tin từ `Charts tracks`

### Hàm `extract_charts_data`

In [ ]:
async def extract_charts_data(url, client_id):
    results = await get_id_from_collection(url, client_id, -1, 'track')
    return results

Đây là 1 `corotine`.

Hàm lấy tất cả các `id` của các `track` với từng `URL` cho trước. `URL` sẽ thay đổi rất nhiều ở hàm gọi hàm này do tính chất của `API`.

- `url`: url cần request
- `client_id`: `id` của `client` thực hiện request.

### Hàm `get_charts`

In [ ]:
async def get_charts(client_id):
    rootLogger.debug("STARTING... get SoundCloud's charts items...")
    kind_options = ['trending', 'top']
    genre_options = [
        'soundcloud:genres:all-music',
        'soundcloud:genres:all-audio',
        'soundcloud:genres:alternativerock',
        'soundcloud:genres:ambient',
        'soundcloud:genres:classical',
        'soundcloud:genres:country',
        'soundcloud:genres:danceedm',
        'soundcloud:genres:dancehall',
        'soundcloud:genres:deephouse',
        'soundcloud:genres:disco',
        'soundcloud:genres:drumbass',
        'soundcloud:genres:dubstep',
        'soundcloud:genres:electronic',
        'soundcloud:genres:folksingersongwriter',
        'soundcloud:genres:hiphoprap',
        'soundcloud:genres:house',
        'soundcloud:genres:indie',
        'soundcloud:genres:jazzblues',
        'soundcloud:genres:latin',
        'soundcloud:genres:metal',
        'soundcloud:genres:piano',
        'soundcloud:genres:pop',
        'soundcloud:genres:rbsoul',
        'soundcloud:genres:reggae',
        'soundcloud:genres:reggaeton',
        'soundcloud:genres:rock',
        'soundcloud:genres:soundtrack',
        'soundcloud:genres:techno',
        'soundcloud:genres:trance',
        'soundcloud:genres:trap',
        'soundcloud:genres:triphop',
        'soundcloud:genres:world',
        'soundcloud:genres:audiobooks',
        'soundcloud:genres:business',
        'soundcloud:genres:comedy',
        'soundcloud:genres:entertainment',
        'soundcloud:genres:learning',
        'soundcloud:genres:newspolitics',
        'soundcloud:genres:religionspirituality',
        'soundcloud:genres:science',
        'soundcloud:genres:sports',
        'soundcloud:genres:storytelling',
        'soundcloud:genres:technology'
    ]
    charts_tracks = []
    results = []
    # tasks = []
    for kind_option in kind_options:
        for genre_option in genre_options:
            url = f'https://api-v2.soundcloud.com/charts?kind={kind_option}&genre={genre_option}&client_id={client_id}&linked_partitioning=1&limit=100'
            charts_tracks.extend(await extract_charts_data(url, client_id))

    charts_tracks = list(set(charts_tracks))

    rootLogger.info(f"FOUND {len(charts_tracks)} charts tracks")

    for track_id in charts_tracks:
        ti = await track_info(track_id, client_id)
        if ti:
            results.append(ti)
            rootLogger.info(f"COMPLETED {len(results)} / {len(charts_tracks)} charts tracks")
        else:
            rootLogger.info(f"COMPLETED 1 charts track")

    rootLogger.info(f"FINISHED got {len(results)} charts tracks")
    return results

Đây là 1 `coroutine`.

Hàm lấy thông tin từ trang [Charts tracks](https://soundcloud.com/charts). Do tính chất đặc thù của `API`, tồn tại nhiều `kind` và `genre` khác nhau, tổ hợp giữa chúng sẽ là các `URL` khác nhau để request.

Các bước chạy như sau:
1. Với từng tổ hợp nói trên, dựa trên `API url` để tìm hết tất cả `id` của `url` này.
2. Với từng `id` tìm được nói trên, gọi làm `track_info`.

Số lượng trả về của hàm này thường khá lớn (~7000), đây cũng là hàm chạy lâu nhất, trong các hàm lấy kết quả không phụ thuộc `keyword`.

- `client_id`: là `id` của `client` thực hiện request.
- `get_charts`: trả về `list` chứa các `dictionary` kết quả.

## Nhóm hàm lấy thông tin từ `discover tracks and playlists` 

### Hàm `get_discover_id`

In [ ]:
async def get_discover_id(client_id):
    rootLogger.debug(f"STARTING... getting discover tracks and playlists")
    tracks = []
    playlists = []
    url = f'https://api-v2.soundcloud.com/mixed-selections?client_id={client_id}&limit=100&linked_partitioning=1'
    response = await request_url(url)
    if not response:
        return tracks, playlists
    collections = response['collection']
    for collection in collections:
        if 'charts' in collection['urn']:
            for no_id_playlist in collection['items']["collection"]:
                for track in no_id_playlist['tracks']:
                    tracks.append(track['id'])
        elif 'playlist' in collection['urn']:
            for playlist in collection['items']['collection']:
                playlists.append(playlist['id'])
        else:
            raise Exception("Unknown type")
    return list(set(tracks)), list(set(playlists))

Đây là 1 `coroutine`.

Hàm lấy tất cả các `id` liên quan trong trang [Discover tracks and playlists](https://soundcloud.com/discover)

Bao gồm cả `id` thuộc về `track` và `id` thuộc về `playlist`
- `client_id`: là `id` của `client` thực hiện request.
- `get_discover_id`: trả về 2 `list`, đó là `list các track` và `list các playlist` trong trang này. 

### Hàm `get_discover_tracks`

In [ ]:

async def get_discover_tracks(tracks, client_id, track_results):
    for track_id in tracks:
        ti = await track_info(track_id, client_id)
        if ti:
            track_results.append(ti)
            rootLogger.info(f"COMPLETED {len(track_results)} / {len(tracks)} discover tracks")
        else:
            rootLogger.info(f"ABORTED 1 discover track")
    rootLogger.info(f"FINISHED got {len(track_results)} discover tracks")

Đây là 1 `coroutine`.

Hàm trả về `track_info` cho một `list` các `track_id` truyền vào.
Sở dĩ hàm không trả về là vì `track_results` là 1 `list` được tham chiếu, kết quả trả về sẽ được `append` trực tiếp vào `list` này.

- `tracks`: 1 `list` các `track's id`
- `track_results`: `list` rỗng, khi hết hàm sẽ chứa kết quả gồm các `dictionary` trả về từ `track_info` 
- `client_id`: `id` của `client` thực hiện request

### Hàm `get_discover_playlists`

In [ ]:
async def get_discover_playlists(playlists, client_id, playlist_results):
    for playlist_id in playlists:
        pi = await playlist_info(playlist_id, client_id)
        if pi:
            playlist_results.append(pi)
            rootLogger.info(f"COMPLETED {len(playlist_results)} / {len(playlists)} discover playlist")
        else:
            rootLogger.info(f"ABORTED 1 discover playlist")
    rootLogger.info(f"FINISHED got {len(playlist_results)} discover playlists")

Đây là 1 `coroutine`.

Hàm trả về `playlist_info` cho một `list` các `playlist_id` truyền vào.
Sở dĩ hàm không trả về là vì `playlists_results` là 1 `list` được tham chiếu, kết quả trả về sẽ được `append` trực tiếp vào `list` này.

- `playlists`: 1 `list` các `playlist's id`
- `playlist_results`: `list` rỗng, khi hết hàm sẽ chứa kết quả gồm các `dictionary` trả về từ `playlist_info` 
- `client_id`: `id` của `client` thực hiện request

### Hàm `get_discover`

In [ ]:
async def get_discover(client_id):
    tracks, playlists = await get_discover_id(client_id)
    rootLogger.info(f"FOUND {len(tracks)} discover tracks")
    rootLogger.info(f"FOUND {len(playlists)} discover playlist")

    track_results = []
    playlist_results = []

    await asyncio.gather(
        get_discover_tracks(tracks, client_id, track_results),
        get_discover_playlists(playlists, client_id, playlist_results)
    )
    return track_results, playlist_results


Đây là 1 `coroutine`.

Hàm sử dụng các hàm trên (cùng nhóm hàm), để tiến hành lấy kết quả từ trang [Discover tracks and playlists](https://soundcloud.com/discover) của `SoundCloud`.

Hàm thông qua các bước:
1. lấy `id` của `tracks` và `playlists` liên quan qua hàm `get_discover_id`
2. Chạy 2 task song song bằng `asyncio.gather` để thu thập dữ liệu bằng hàm `get_discover_tracks` và `get_discover_playlists`

- `client_id`: `id` của `client` thực hiện request
- `get_discover`: trả về 2 `list`, mỗi `list` chứa nhiều `dictionary` của từng nhóm `track` và `playlist` tương ứng.

# Nhóm hàm thực thi chương trình

## Hàm extract_data



In [ ]:
def extract_data(results):
    tracks = []
    playlists = []
    users = []

    generals_data = results[0]
    query_data = results[1]

    for charts_track in generals_data[0]:
        tracks.append(charts_track)
    for discover_track in generals_data[1][0]:
        tracks.append(discover_track)
    for discover_playlist in generals_data[1][1]:
        playlists.append(discover_playlist)
    for featured_track in generals_data[2]:
        tracks.append(featured_track)

    for query_results in query_data:
        for query_playlist in query_results[0]:
            playlists.append(query_playlist)
        for query_user in query_results[1]:
            users.append(query_user)
        for query_track in query_results[2]:
            tracks.append(query_track)
    tracks = list({track['id']: track for track in tracks}.values())
    playlists = list({playlist['id']: playlist for playlist in playlists}.values())
    users = list({user['id']: user for user in users}.values())
    return tracks, playlists, users

Hàm tái cấu trúc, loại bỏ trùng lặp dữ liệu nhận được, trả về dữ liệu cuối cùng để lưu.

`asyncio.gather` trả về các giá trị kết quả trong 1 `list`, mà ta lại chạy nhiều hàm cùng lúc, đó là lí do `extract_data` là cần thiết để rút trích dữ liệu ra từ các tiến trình.

Sau khi rút trích hoàn tất, ta tiến hành loại bỏ các dữ liệu trùng lặp và trả về kết quả.

Kết quả này đã sẵn sàng để lưu vào file `csv`.

Ta có thể tóm gọn trong các bước:
1. Trích kết quả chi tiết
2. Loại bỏ trùng lặp dữ liệu

- `results`: có được từ hàm `main`
- `extract_data`: trả về 3 `list` các `dictionary` thuộc loại khác nhau, sẵn sàng để tiến hành lưu

## Hàm main


In [ ]:
async def main():
    client_id = 'qgbUmYdRbdAL2R1aLbVCgwzC7mvh8VKv'

    keyword_list = ['adele', 'justin', 'adam', 'imagine dragon']
    query_tasks = []
    for keyword in keyword_list:
        tasks = asyncio.gather(
            get_query_item(QueryType.PLAYLISTS, keyword, client_id, 1000),
            get_query_item(QueryType.USERS, keyword, client_id, 1000),
            get_query_item(QueryType.TRACKS, keyword, client_id, 1000),
        )
        query_tasks.append(tasks)
    query_tasks = asyncio.gather(*query_tasks)
    general_tasks = asyncio.gather(
        get_charts(client_id),
        get_discover(client_id),
        get_featured(client_id)
    )
    start_time = datetime.now()
    rootLogger.info('##############################')
    rootLogger.info("CRAWLING DATA BEGIN, DON'T PANIC IF IT LOOK STUCK...")
    rootLogger.info('##############################')

    results = await asyncio.gather(general_tasks, query_tasks)
    tracks, playlists, users = extract_data(results)
    end_time = datetime.now()
    total = end_time - start_time
    total_seconds = int(total.total_seconds())
    hours, remainder = divmod(total_seconds, 60 * 60)
    minutes, seconds = divmod(remainder, 60)
    rootLogger.info('##############################')
    rootLogger.info("DATA CRAWLING COMPLETED")
    rootLogger.info('##############################')
    rootLogger.info('total time: {} hrs {} mins {} secs'.format(hours, minutes, seconds))
    rootLogger.info('result: ')
    rootLogger.info(f'\tTracks: {len(tracks)} items')
    rootLogger.info(f'\tPlaylists: {len(playlists)} items')
    rootLogger.info(f'\tUsers: {len(users)} items')
    rootLogger.info('##############################')
    rootLogger.info("saving data...")

    save_file('tracks_file.csv', tracks)
    save_file('playlists_file.csv', playlists)
    save_file('users_file.csv', users)
    rootLogger.info("DONE")




Hàm chạy tất cả các hàm trên, cùng với một số tác vụ nhỏ như tính thời gian chạy, in ra số kết quả thu thập được.

Trong `main` ta tiến hành gom nhóm các tiến trình lại với nhau bằng `asyncio.gather` để thực hiện song song các tiến trình.

`keyword_list` chứa các `keyword` dùng để `query`, với mỗi `keyword` trong `keyword_list`, ta tiến hành query cho đủ 3 loại là `USER`, `PLAYLIST`, `TRACKS`

Cùng với đó, ta cũng cho chạy các hàm lấy dữ liệu không phụ thuộc vào keyword, cụ thể ở đây là các hàm: `get_charts`, `get_discover`, `get_featured`.

Chương trình sẽ tiến hành chạy và đưa ra kết quả, kết quả này sau đó sẽ được trích, lọc trong hàm `extract_data` và lưu vào 3 file các keert quả tương ứng.

## Chạy chương trình

### Window

In [ ]:
# asyncio.get_event_loop().run_until_complete(main())

Vì một số lỗi của thư viện `aiohttp` khi chạy trên window nói chung, ta sẽ thực thi chương trình bằng đoạn lệnh trên

### Linux

In [ ]:
# asyncio.run(main())

Đoạn lệnh thực thi này được test trên máy `Ubuntu` và chạy không lỗi, những đoạn lệnh khác có thể chạy ở đây, nhưng nhóm chưa có dịp thử nghiệm.

### Jupyter

In [ ]:
# await main()

Đoạn lệnh trên được một bạn trong nhóm tìm hiểu và khuyến nghị chạy trên môi trường jupyter notebook nói chung